In [ ]:
import jax.numpy as jnp
from jax import jit

from genjax import ChoiceMap as Chm
from genjax import Selection as Sel
from genjax import beta, flip, gen
from genjax.edits import HMC


# Create a generative function.
@gen
def beta_bernoulli(α, β):
    p = beta(α, β) @ "p"
    v = flip(p) @ "v"
    return v


def exact_posterior_mean(obs, α, β):
    return (α + obs) / (α + β + 1)


# Implements HMC-within-SIR:
# create a trace, edit it with HMC, resample.
@jit
def inference_via_editing_traces(obs, α, β):
    (tr, lws) = beta_bernoulli.importance_k(50)(
        Chm.d({"v": obs}),  # constraint: "v" -> True
        (α, β),  # (α, β)
    )
    (tr, lws_, *_) = tr.edit_k(
        # run a single step of HMC for "p" with eps=1e-3.
        HMC(Sel.at["p"], jnp.array(1e-3)),
    )
    (tr, _) = tr.resample_k(lws + lws_)
    return jnp.mean(tr["p"])


α, β = 2.0, 2.0
obs = True
(
    exact_posterior_mean(obs, α, β),
    inference_via_editing_traces(obs, α, β),
)